# Data Aggregation and Group Operations

In [ ]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 10.1. GroupBy Mechanics

### 10.1.0

To get started, here is a small tabular dataset as a DataFrame:

In [1]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,key1,key2,data1,data2
0,a,one,0.268822,1.832557
1,a,two,0.029415,-1.251991
2,b,one,-0.507335,0.627406
3,b,two,0.322260,0.246063
4,a,one,-0.046475,-0.349921


Suppose you wanted to *compute the mean of the data1 column* using the *labels from key1*.<br>
There are a number of ways to do this.<br>
* One is to access data1 and call groupby with the column (a Series) at key1:

In [2]:
grouped = df['data1'].groupby(df['key1'])
grouped

This grouped variable is now a GroupBy object. It has not actually computed anything yet except for some intermediate data about the group key df['key1']. The idea is that this object has all of the information needed to then apply some operation to each of the groups.<br>

For example, to compute group means we can call the GroupBy’s mean method:

In [3]:
grouped.mean()

key1
a    0.083921
b   -0.092538
Name: data1, dtype: float64

Later, I’ll explain more about what happens when you call .mean(). The important thing here is that `the data (a Series) has been aggregated according to the group key`, producing a new Series that is now indexed by the unique values in the key1 column. The result index has the name 'key1' because the DataFrame column df['key1'] did.<br>

If instead we had passed multiple arrays as a list, we’d get something different:

In [6]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.111174
      two     0.029415
b     one    -0.507335
      two     0.322260
Name: data1, dtype: float64

Here we grouped the data using two keys (@P:key 1 + key 2 above), and the resulting Series now has a hierarchical index consisting of the unique pairs of keys observed:

In [7]:
means.unstack()

key2,one,two
key1,,
a,0.111174,0.029415
b,-0.507335,0.322260


In this example, the group keys are all Series, though they could be any arrays of the right length:

In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

California  2005    0.029415
            2006   -0.507335
Ohio        2005    0.295541
            2006   -0.046475
Name: data1, dtype: float64

Frequently the grouping information is found in the same DataFrame as the data you want to work on. In that case, you can pass column names (whether those are strings, numbers, or other Python objects) as the group keys:

In [9]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.083921,0.076882
b,-0.092538,0.436734


In [10]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.111174  0.741318
     two   0.029415 -1.251991
b    one  -0.507335  0.627406
     two   0.322260  0.246063

You may have noticed in the first case df.groupby('key1').mean() that there is no key2 column in the result. *Because df['key2'] is not numeric data*, it is said to be a `nuisance column`, which is therefore excluded from the result. `By default, all of the numeric columns are aggregated`, though it is possible to filter down to a subset, as you’ll see soon.<br>
Regardless of the objective in using groupby, a generally useful GroupBy method is size, which returns a Series containing group sizes:

In [11]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

Take note that any missing values in a group key will be excluded from the result.

### 10.1.1. Iterating Over Groups

The `GroupBy object supports iteration`, generating a sequence of 2-tuples containing the group name along with the chunk of data. Consider the following:

In [12]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.268822  1.832557
1    a  two  0.029415 -1.251991
4    a  one -0.046475 -0.349921
b
  key1 key2     data1     data2
2    b  one -0.507335  0.627406
3    b  two  0.322260  0.246063


In the case of multiple keys, the first element in the tuple will be a tuple of key values:

In [13]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.268822  1.832557
4    a  one -0.046475 -0.349921
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.029415 -1.251991
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.507335  0.627406
('b', 'two')
  key1 key2    data1     data2
3    b  two  0.32226  0.246063


Of course, you can choose to do whatever you want with the pieces of data. *A recipe you may find useful* is `computing a dict of the data pieces as a one-liner`:

In [15]:
pieces = dict(list(df.groupby('key1')))
pieces
# pieces['b']

{'a':   key1 key2     data1     data2
 0    a  one  0.268822  1.832557
 1    a  two  0.029415 -1.251991
 4    a  one -0.046475 -0.349921,
 'b':   key1 key2     data1     data2
 2    b  one -0.507335  0.627406
 3    b  two  0.322260  0.246063}

By default groupby groups on `axis=0`, but you `can group on any of the other axes`. For example, we could group the columns of our example df here by dtype like so:

In [18]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [21]:
grouped = df.groupby(df.dtypes, axis=1)
grouped

In [24]:
#We can print out the groups like so:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.268822  1.832557
1  0.029415 -1.251991
2 -0.507335  0.627406
3  0.322260  0.246063
4 -0.046475 -0.349921
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 10.1.2. Selecting a Column or Subset of Columns

Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation. This means that:

In [25]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

are `syntactic sugar` for:

In [ ]:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

Especially for large datasets, it may be desirable to aggregate only a few columns.<br>
For example, in the preceding dataset, to *compute means for just the data2 column* and get the result as a DataFrame, we could write:

In [26]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.741318
     two  -1.251991
b    one   0.627406
     two   0.246063

`The object returned by this indexing operation is a grouped DataFrame` if a *list or array* is passed or` a grouped Series` if only a *single column name is passed* as a scalar:

In [27]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [28]:
s_grouped.mean()

key1  key2
a     one     0.741318
      two    -1.251991
b     one     0.627406
      two     0.246063
Name: data2, dtype: float64

### 10.1.3. Grouping with Dicts and Series

Grouping information may exist in a form other than an array. Let’s consider another example DataFrame:

In [29]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,a,b,c,d,e
Joe,-0.073964,0.187162,0.812466,-1.576396,-0.526108
Steve,1.415394,-1.298256,0.246182,0.549330,-0.622651
Wes,-0.094032,NaN,NaN,-0.358496,2.053419
Jim,0.299956,0.586976,0.191228,-1.628495,1.487402
Travis,0.834516,0.143266,0.854893,1.696099,-1.215560


Now, suppose I have a group correspondence for the columns and want to sum together the columns by group:

In [30]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}
mapping

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

Now, you could construct an array from this dict to pass to groupby, but instead we can just pass the dict (I included the key 'f' to highlight that unused grouping keys are OK):

In [31]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-0.763930,-0.412910
Steve,0.795512,-0.505512
Wes,-0.358496,1.959387
Jim,-1.437268,2.374334
Travis,2.550992,-0.237777


`The same functionality holds for Series`, which can be viewed as a *fixedsize mapping*:

In [32]:
map_series = pd.Series(mapping)
map_series

<IPython.core.display.Javascript object>

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [34]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 10.1.4. Grouping with Functions

Using Python functions is a more generic way of defining a group mapping compared with a dict or Series. `Any function passed as a group key will be called once per index value, with the return values being used as the group names`.<br>
More concretely, consider the example DataFrame from the previous section, which has people’s first names as index values. Suppose you wanted to group by the length of the names; while you could compute an array of string lengths, it’s simpler to just pass the len function:

In [35]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.131959,0.774138,1.003694,-3.563387,3.014714
5,1.415394,-1.298256,0.246182,0.549330,-0.622651
6,0.834516,0.143266,0.854893,1.696099,-1.215560


Mixing functions with arrays, dicts, or Series is not a problem as everything gets converted to arrays internally:

In [37]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()
#@P 20210901: not understand how key_list function in this syntax

a         b         c         d         e
3 one -0.094032  0.187162  0.812466 -1.576396 -0.526108
  two  0.299956  0.586976  0.191228 -1.628495  1.487402
5 one  1.415394 -1.298256  0.246182  0.549330 -0.622651
6 two  0.834516  0.143266  0.854893  1.696099 -1.215560

### 10.1.5. Grouping by Index Levels

A final convenience for hierarchically indexed datasets is the ability to aggregate using one of the levels of an axis index.<br>
Let’s look at an example:

In [38]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.919049  1.188499 -1.587444 -1.047209 -1.323575
1      0.157714  0.325778 -0.282669  0.710322 -0.014956
2      1.039628 -1.256641 -0.303179 -0.223144 -0.735214
3      0.334560 -0.532842 -0.386367 -0.752408 -1.586254

* *To group by level*, pass the level number or name using the level keyword:

In [39]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2. Data Aggregation

In [ ]:
df
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

In [ ]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

In [ ]:
grouped.describe()

### 10.2.1. Column-Wise and Multiple Function Application

In [ ]:
tips = pd.read_csv('examples/tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

In [ ]:
grouped = tips.groupby(['day', 'smoker'])

In [ ]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

In [ ]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

In [ ]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

In [ ]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

In [ ]:
result['tip_pct']

In [ ]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

In [ ]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

### 10.2.2. Returning Aggregated Data Without Row Indexes

In [ ]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

## 10.3. Apply: General split-apply-combine

In [ ]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

In [ ]:
tips.groupby('smoker').apply(top)

In [ ]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

In [ ]:
result = tips.groupby('smoker')['tip_pct'].describe()
result
result.unstack('smoker')

f = lambda x: x.describe()
grouped.apply(f)

### 10.3.1. Suppressing the Group Keys

In [ ]:
tips.groupby('smoker', group_keys=False).apply(top)

### 10.3.2. Quantile and Bucket Analysis

In [ ]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

In [ ]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

In [ ]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

### 10.3.3. Example: Filling Missing Values with Group-Specific       Values

In [ ]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s
s.fillna(s.mean())

In [ ]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

In [ ]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data
data.groupby(group_key).mean()

In [ ]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

In [ ]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

### 10.3.4. Example: Random Sampling and Permutation

In [ ]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [ ]:
deck[:13]

In [ ]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

In [ ]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

In [ ]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

### 10.3.5. Example: Group Weighted Average and Correlation

In [ ]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

In [ ]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

In [ ]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True,
                       index_col=0)
close_px.info()
close_px[-4:]

In [ ]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [ ]:
rets = close_px.pct_change().dropna()

In [ ]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

In [ ]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

### 10.3.6. Example: Group-Wise Linear Regression

In [ ]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [ ]:
by_year.apply(regress, 'AAPL', ['SPX'])

## 10.4. Pivot Tables and Cross-Tabulation

In [ ]:
tips.pivot_table(index=['day', 'smoker'])

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

### Cross-Tabulations: Crosstab

In [ ]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [ ]:
data

In [ ]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

In [ ]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

In [ ]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

## 10.5. Conclusion